In [1]:
from keras import backend as K
from keras.models import load_model
from keras.preprocessing import image
from keras.optimizers import Adam
from imageio import imread
import numpy as np
import matplotlib.patches as patches
from matplotlib import pyplot as plt

# from models.keras_ssd300 import ssd_300
# from models.keras_ssd300_vgg_13_bn import ssd_300
# from models.keras_ssd300_mobilenetv1_imagenet import ssd_300
# from models.keras_ssd300_mobilenetv1_imagenet import ssd_300
# from models.keras_ssdlite320_mobilenetv1 import ssd_300
# from models.keras_ssdlite320_mobilenetv2 import ssd_300
# from models.keras_ssdlite320_mobilenetv2_conv13_17 import ssd_300
from models.keras_ssdlite320PPN_mobilenetv2 import ssd_300

from models.keras_ssd512 import ssd_512
from keras_loss_function.keras_ssd_loss import SSDLoss
from keras_layers.keras_layer_AnchorBoxes import AnchorBoxes
from keras_layers.keras_layer_DecodeDetections import DecodeDetections
from keras_layers.keras_layer_DecodeDetectionsFast import DecodeDetectionsFast
from keras_layers.keras_layer_L2Normalization import L2Normalization

from ssd_encoder_decoder.ssd_output_decoder import decode_detections, decode_detections_fast

from data_generator.object_detection_2d_data_generator import DataGenerator
from data_generator.object_detection_2d_photometric_ops import ConvertTo3Channels
from data_generator.object_detection_2d_geometric_ops import Resize
from data_generator.object_detection_2d_misc_utils import apply_inverse_transforms

import tensorflow as tf
import keras as keras

Using TensorFlow backend.
/home/aldo/Documents/ssd/data_generator/object_detection_2d_data_generator.py:43: UserWarning: 'BeautifulSoup' module is missing. The XML-parser will be unavailable.
  warnings.warn("'BeautifulSoup' module is missing. The XML-parser will be unavailable.")


In [2]:
# path_models = '/home/aldo/Documents/data-cic/tflite_models/vgg_13_bn'
# path_models = '/home/aldo/Documents/data-cic/tflite_models/vgg_16_2'
# path_models = '/home/aldo/Documents/data-cic/tflite_models/vgg_16_20'
# path_models = '/home/aldo/Documents/data-cic/tflite_models/vgg_16_20'
# path_models = '/home/aldo/Documents/data-cic/tflite_models/mobilenetv1'
# path_models = '/home/aldo/Documents/data-cic/tflite_models/ssdlitemnetv1'
# path_models = '/home/aldo/Documents/data-cic/tflite_models/13_17'
path_models = '/home/aldo/Documents/data-cic/tflite_models/PPN'

# Load the trained model

In [3]:
def load_model(model_path, height, width):
    # 1: Build the Keras model
    K.clear_session() # Clear previous models from memory.

#     scales = [0.1, 0.2, 0.37, 0.54, 0.71, 0.88, 1.05] # pascal
#     scales = [0.07, 0.15, 0.33, 0.51, 0.69, 0.87, 1.05] # coco
    scales = [0.15, 0.33, 0.47, 0.61, 0.76, 0.90, 1.05] # mobilenet

#     scales = [0.2, 0.61, 1.05]

    
#     swap_channels = [2, 1, 0]
    swap_channels = False
#     mean = [123, 117, 104]
#     mean = [127, 127, 127]
    mean = [1., 1., 1.]
    divide_by_stddev = [127.5, 127.5, 127.5]
#     divide_by_stddev = None
    
#     steps = [8, 16, 32, 64, 100, 300]
    steps = [16, 30, 60, 100, 150, 300]
#     steps = [16, 32, 64, 107, 160, 320]
#     steps = [16, 32]
#     classes = 20
    classes = 1
    
    if height == 300 or height == 320:
        model = ssd_300(image_size=(height, width, 3),
                        n_classes=classes,
                        alpha = 1.0,
                        expansion = 1, 
                        mode='inference',
#                         l2_regularization=0.0005,
                        scales=scales,
#                         aspect_ratios_per_layer=[[1.0, 2.0, 0.5],
#                                                  [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
#                                                  [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
#                                                  [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
#                                                  [1.0, 2.0, 0.5],
#                                                  [1.0, 2.0, 0.5]],
                        aspect_ratios_per_layer=[[1.0, 0.5, 2.0/3.0],
                                                 [1.0, 0.5, 2.0/3.0, 1.0/3.0, 3.0/4.0],
                                                 [1.0, 0.5, 2.0/3.0, 1.0/3.0],
                                                 [1.0, 2.0, 2.0/3.0, 1.0/3.0],
                                                 [1.0, 2.0, 2.0/3.0, 1.0/3.0],
                                                 [1.0, 2.0, 2.0/3.0, 1.0/3.0]],
                        two_boxes_for_ar1=True,
                        steps=steps,
                        offsets=[0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
#                         offsets=[0.5, 0.5],
                        clip_boxes=False,
                        variances=[0.1, 0.1, 0.2, 0.2],
                        normalize_coords=True,
                        subtract_mean=mean,
                        swap_channels=False, # set to false otherwise funtion not implemented
                        confidence_thresh=0.15,
                        iou_threshold=0.45,
                        top_k=200,
                        nms_max_output_size=400,
                        divide_by_stddev=divide_by_stddev)
    elif height == 512:
        model = ssd_512(image_size=(height, width, 3),
                n_classes=20,
                mode='inference',
                l2_regularization=0.0005,
                scales=[0.07, 0.15, 0.3, 0.45, 0.6, 0.75, 0.9, 1.05], # The scales for MS COCO are [0.04, 0.1, 0.26, 0.42, 0.58, 0.74, 0.9, 1.06]
                aspect_ratios_per_layer=[[1.0, 2.0, 0.5],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5],
                                         [1.0, 2.0, 0.5]],
               two_boxes_for_ar1=True,
               steps=[8, 16, 32, 64, 128, 256, 512],
               offsets=[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
               clip_boxes=False,
               variances=[0.1, 0.1, 0.2, 0.2],
               normalize_coords=True,
               subtract_mean=[123, 117, 104],
               swap_channels=False,
               confidence_thresh=0.15,
               iou_threshold=0.45,
               top_k=200,
               nms_max_output_size=400)

    # 2: Load the trained weights into the model.
    # TODO: Set the path of the trained weights.
    model.load_weights(model_path, by_name=True)

    # 3: Compile the model so that Keras won't complain the next time you load it.

    adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

    ssd_loss = SSDLoss(neg_pos_ratio=3, alpha=1.0)

    model.compile(optimizer=adam, loss=ssd_loss.compute_loss)
    
    return model

In [4]:
def load_graph(frozen_graph_filename):
    '''
    Load a frozen graph
    frozen_grapth_filename: path in the disk
    return: tensorflow graph
    '''
    # We load the protobuf file from the disk and parse it to retrieve the 
    # unserialized graph_def
    with tf.gfile.GFile(frozen_graph_filename, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        
    # Then, we import the graph_def into a new Graph and returns it 
    with tf.Graph().as_default() as graph:
        # The name var will prefix every op/nodes in your graph
        # Since we load everything in a new graph, this is not needed
        tf.import_graph_def(graph_def, name="prefix")
    return graph

def save_graph(name):
    '''
    Save grapth to load with tensorboard
    '''
    graph = load_graph(name)

    # save data from a grapth to display on tensorboard
    # tensorboard --logdir=wholepath
    # the path has to be absolute
    with tf.Session(graph=graph) as sess:
        tf.summary.FileWriter('../logs_ssd', sess.graph)

def show_operations_graph(graph):
    '''
    Print operations contained in a graph
    '''
    for op in graph.get_operations():
        print(op.name)

In [5]:
# Load models and weights
# model_path = '../weights/pascal_voc/pascal_voc_07_12_coco_300x300.h5'
# model_path = '../weights/models/ssd300_sgd/ssd300_fine_tunning_epoch-89_loss-2.6171_val_loss-3.0580.h5'
# model_path = '../weights/models/ssd300_adam_vgg16/ssd300_pascal_cic.h5'
# model_path = '../weights/models/ssd300_adam_vgg13_bn/ssd300_vgg_13_bn_pascal_cic.h5'
# model_path = '../weights/models/ssd_mobilenetv1/ssd300_mobilenetv1_imagenet_cic.h5'
# model_path = '../weights/models/ssdlite320_mobilenetv1/ssdlite320_mobilenetv1_pascal_cic.h5'

# model_path = '../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_0.35_exp_1.h5'
# model_path = '../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_f1_0.35_exp_1.h5'
model_path = '../weights/models/new_net/new_share_conv_conf_cic.h5'

model = load_model(model_path, 300, 300)

In [6]:
# Save models as .pbtxt or pb
tf.train.write_graph(K.get_session().graph, path_models, "ssd_model.pbtxt", as_text=True)
tf.train.write_graph(K.get_session().graph, path_models, "ssd_model.pb", as_text=False)

'/home/aldo/Documents/data-cic/tflite_models/PPN/ssd_model.pb'

In [7]:
save_graph(path_models + '/ssd_model.pb')

In [8]:
# Add ops to save and restore all the variables.
saver = tf.train.Saver()
sess = keras.backend.get_session()
save_path = saver.save(sess, path_models + "/ssd_model.ckpt")

## Freeze the Graph
In order to convert the graph to tflite we need to freeze it, in this case we'll use the tools provided by tensorflow to do so.

We need to specify the graph, the checkpoints, the output name.

In [9]:
# Freeze the graph
import tensorflow as tf
from tensorflow.python.tools import freeze_graph

input_graph_path = path_models + '/ssd_model.pb'
checkpoint_path = path_models + '/ssd_model.ckpt'
input_saver_def_path = ""
input_binary = True
output_node_names = "decoded_predictions/loop_over_batch/TensorArrayStack/TensorArrayGatherV3"
restore_op_name = "save/restore_all"
filename_tensor_name = "save/Const:0"
output_frozen_graph_name = path_models + '/ssd_frozen_model.pb'
clear_devices = True


freeze_graph.freeze_graph(input_graph_path, input_saver_def_path,
                          input_binary, checkpoint_path, output_node_names,
                          restore_op_name, filename_tensor_name,
                          output_frozen_graph_name, clear_devices, "")

INFO:tensorflow:Restoring parameters from /home/aldo/Documents/data-cic/tflite_models/PPN/ssd_model.ckpt
INFO:tensorflow:Froze 282 variables.
INFO:tensorflow:Converted 282 variables to const ops.


## Optimize for inference

Optimize for inference will help us to remove those parts of the graph that were used during training.

We need the input and output names and the frozen model.

In [10]:
!python3 -m tensorflow.python.tools.optimize_for_inference \
--input=./extra_files/models/ssd_frozen_model.pb \
--output=./extra_files/models/ssd_opt_model.pb \
--frozen_graph=True --input_names=input_1 \
--output_names=predictions/concat

Input graph file './extra_files/models/ssd_frozen_model.pb' does not exist!


In [10]:
# !python3 -m tensorflow.python.tools.optimize_for_inference \
# --input=/home/aldo/Documents/data-cic/tflite_models/vgg_16_20/ssd_frozen_model.pb \
# --output=/home/aldo/Documents/data-cic/tflite_models/vgg_16_20/ssd_opt_model.pb \
# --frozen_graph=True --input_names=input_1 \
# --output_names=predictions/concat

# !python3 -m tensorflow.python.tools.optimize_for_inference \
# --input=/home/aldo/Documents/data-cic/tflite_models/vgg_16_2/ssd_frozen_model.pb \
# --output=/home/aldo/Documents/data-cic/tflite_models/vgg_16_2/ssd_opt_model.pb \
# --frozen_graph=True --input_names=input_1 \
# --output_names=predictions/concat

# !python3 -m tensorflow.python.tools.optimize_for_inference \
# --input=/home/aldo/Documents/data-cic/tflite_models/vgg_13_bn/ssd_frozen_model.pb \
# --output=/home/aldo/Documents/data-cic/tflite_models/vgg_13_bn/ssd_opt_model.pb \
# --frozen_graph=True --input_names=input_1 \
# --output_names=predictions/concat

# !python3 -m tensorflow.python.tools.optimize_for_inference \
# --input=/home/aldo/Documents/data-cic/tflite_models/mobilenetv1/ssd_frozen_model.pb \
# --output=/home/aldo/Documents/data-cic/tflite_models/mobilenetv1/ssd_opt_model.pb \
# --frozen_graph=True --input_names=input_1 \
# --output_names=predictions/concat

# !python3 -m tensorflow.python.tools.optimize_for_inference \
# --input=/home/aldo/Documents/data-cic/tflite_models/ssdlitemnetv1/ssd_frozen_model.pb \
# --output=/home/aldo/Documents/data-cic/tflite_models/ssdlitemnetv1/ssd_opt_model.pb \
# --frozen_graph=True --input_names=input_1 \
# --output_names=predictions/concat

# !python3 -m tensorflow.python.tools.optimize_for_inference \
# --input=/home/aldo/Documents/data-cic/tflite_models/ssdlitemnetv2/ssd_frozen_model.pb \
# --output=/home/aldo/Documents/data-cic/tflite_models/ssdlitemnetv2/ssd_opt_model.pb \
# --frozen_graph=True --input_names=input_1 \
# --output_names=predictions/concat

# !python3 -m tensorflow.python.tools.optimize_for_inference \
# --input=/home/aldo/Documents/data-cic/tflite_models/13_17/ssd_frozen_model.pb \
# --output=/home/aldo/Documents/data-cic/tflite_models/13_17/ssd_opt_model.pb \
# --frozen_graph=True --input_names=input_1 \
# --output_names=predictions/concat

# !python3 -m tensorflow.python.tools.optimize_for_inference \
# --input=/home/aldo/Documents/data-cic/tflite_models/trade-off/ssd_frozen_model.pb \
# --output=/home/aldo/Documents/data-cic/tflite_models/trade-off/ssd_opt_model.pb \
# --frozen_graph=True --input_names=input_1 \
# --output_names=predictions/concat

!python3 -m tensorflow.python.tools.optimize_for_inference \
--input=/home/aldo/Documents/data-cic/tflite_models/PPN/ssd_frozen_model.pb \
--output=/home/aldo/Documents/data-cic/tflite_models/PPN/ssd_opt_model.pb \
--frozen_graph=True --input_names=input_1 \
--output_names=predictions/concat

# Convert to tflite model
Finally we conver the model to tflite format, we need a frozen model in the same way the input and output name are required, and all the operations in the model need to be conpatible with tflite.

In [122]:
graph_def_file = "/home/aldopedraza/Documentos/models/ssd_frozen_model.pb"
input_arrays = ["input_1"]
output_arrays = ["decoded_predictions/loop_over_batch/TensorArrayStack/TensorArrayGatherV3"]

converter = tf.contrib.lite.TocoConverter.from_frozen_graph(
  graph_def_file, input_arrays, output_arrays, input_shapes={"input_1" : [1, 300, 300, 3]})
tflite_model = converter.convert()
open("/home/aldopedraza/Documentos/models/converted_model.tflite", "wb").write(tflite_model)

In [1]:
# !toco \
#   --graph_def_file=/home/aldopedraza/Documentos/ssd/extra_files/models/ssd_opt_model.pb \
#   --output_file=/home/aldopedraza/Documentos/ssd/extra_files/models/tflite_model.tflite \
#   --input_format=TENSORFLOW_GRAPHDEF --output_format=TFLITE \
#   --inference_type=QUANTIZED_UINT8 \
#   --input_shape="1,512,512,3" \
#   --input_array=input_1 \
#   --output_array=predictions/concat \
#   --std_dev_values=127.5 --mean_value=127.5

!toco \
--output_file=/home/aldopedraza/Downloads/tflite_model.tflite \
--graph_def_file=/home/aldo/Downloads/prueba/ssd_opt_model.pb \
--output_format=TFLITE \
--inference_type=QUANTIZED_UINT8 \
--inference_input_type=QUANTIZED_UINT8 \
--input_arrays=input_1 \
--output_arrays=predictions/concat \
--input_shapes=1,320,320,3 \
--mean_values=128 \
--std_dev_values=128 \
--default_ranges_min=0 \
--default_ranges_max=6

2019-03-06 18:38:53.679403: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2019-03-06 18:38:53.791560: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:964] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-03-06 18:38:53.792072: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1411] Found device 0 with properties: 
name: GeForce GTX 1080 major: 6 minor: 1 memoryClockRate(GHz): 1.7335
pciBusID: 0000:01:00.0
totalMemory: 7.92GiB freeMemory: 6.95GiB
2019-03-06 18:38:53.792088: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1490] Adding visible gpu devices: 0
2019-03-06 18:38:53.981024: I tensorflow/core/common_runtime/gpu/gpu_device.cc:971] Device interconnect StreamExecutor with strength 1 edge matrix:
2019-03-06 18:38:53.981061: I tensorflow/core/common_runtime/gpu/gpu_device.cc:977]      0 

In [1]:
### THIS WORKS
# !tflite_convert --output_file=/home/aldo/Documents/data-cic/tflite_models/vgg_16_20/tflite_vgg16_20.tflite --graph_def_file=/home/aldo/Documents/data-cic/tflite_models/vgg_16_20/ssd_opt_model.pb --input_arrays=input_1 --output_arrays=predictions/concat --input_shapes=1,300,300,3
# !tflite_convert --output_file=/home/aldo/Documents/data-cic/tflite_models/vgg_16_2/tflite_vgg16_2.tflite --graph_def_file=/home/aldo/Documents/data-cic/tflite_models/vgg_16_2/ssd_opt_model.pb --input_arrays=input_1 --output_arrays=predictions/concat --input_shapes=1,300,300,3
# !tflite_convert --output_file=/home/aldo/Documents/data-cic/tflite_models/vgg_13_bn/tflite_vgg13_bn.tflite --graph_def_file=/home/aldo/Documents/data-cic/tflite_models/vgg_13_bn/ssd_opt_model.pb --input_arrays=input_1 --output_arrays=predictions/concat --input_shapes=1,300,300,3
# !tflite_convert --output_file=/home/aldo/Documents/data-cic/tflite_models/mobilenetv1/tflite_mobilenetv1.tflite --graph_def_file=/home/aldo/Documents/data-cic/tflite_models/mobilenetv1/ssd_opt_model.pb --input_arrays=input_1 --output_arrays=predictions/concat --input_shapes=1,300,300,3
# !tflite_convert --output_file=/home/aldo/Documents/data-cic/tflite_models/ssdlitemnetv1/ssdlitemnetv1.tflite --graph_def_file=/home/aldo/Documents/data-cic/tflite_models/ssdlitemnetv1/ssd_opt_model.pb --input_arrays=input_1 --output_arrays=predictions/concat --input_shapes=1,320,320,3
# !tflite_convert --output_file=/home/aldo/Documents/data-cic/tflite_models/ssdlitemnetv2/ssdlitemnetv2.tflite --graph_def_file=/home/aldo/Documents/data-cic/tflite_models/ssdlitemnetv2/ssd_opt_model.pb --input_arrays=input_1 --output_arrays=predictions/concat --input_shapes=1,320,320,3
# !tflite_convert --output_file=/home/aldo/Documents/data-cic/tflite_models/13_17/conv13_17.tflite --graph_def_file=/home/aldo/Documents/data-cic/tflite_models/13_17/ssd_opt_model.pb --input_arrays=input_1 --output_arrays=predictions/concat --input_shapes=1,320,320,3

# !tflite_convert \
# --output_file=/home/aldo/Documents/data-cic/tflite_models/trade-off/tflite_files/alpha_35_exp_1_f1.tflite \
# --graph_def_file=/home/aldo/Documents/data-cic/tflite_models/trade-off/ssd_opt_model.pb \
# --input_arrays=input_1 \
# --output_arrays=predictions/concat \
# --input_shapes=1,320,320,3

!tflite_convert \
--output_file=/home/aldo/Documents/data-cic/tflite_models/PPN/PPN_alpha_1_exp_1_f1.tflite \
--graph_def_file=/home/aldo/Documents/data-cic/tflite_models/PPN/ssd_opt_model.pb \
--input_arrays=input_1 \
--output_arrays=predictions/concat \
--input_shapes=1,300,300,3

2019-03-22 13:29:16.409932: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2019-03-22 13:29:16.509713: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:964] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-03-22 13:29:16.510118: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1411] Found device 0 with properties: 
name: GeForce GTX 1080 major: 6 minor: 1 memoryClockRate(GHz): 1.7335
pciBusID: 0000:01:00.0
totalMemory: 7.92GiB freeMemory: 7.00GiB
2019-03-22 13:29:16.510141: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1490] Adding visible gpu devices: 0
2019-03-22 13:29:16.693400: I tensorflow/core/common_runtime/gpu/gpu_device.cc:971] Device interconnect StreamExecutor with strength 1 edge matrix:
2019-03-22 13:29:16.693439: I tensorflow/core/common_runtime/gpu/gpu_device.cc:977]      0 

## Test model to see if it's compatible with tflite

In order to be sure that the model works correctly what we can do it's to use the interpreter to load the model and test it.

In [2]:
import numpy as np
import tensorflow as tf
from scipy import misc

# Read image
image_test = misc.imread('./image_test.jpg')
image_test = misc.imresize(image_test, size=(300, 300))
misc.imsave('./resize_image.jpg', image_test)
image_test = image_test.reshape(1, 300, 300, 3)

# Load TFLite model and allocate tensors.
# interpreter = tf.contrib.lite.Interpreter(model_path='/home/aldo/Documents/data-cic/tflite_models/vgg_16_20/tflite_vgg16_20.tflite')
# interpreter = tf.contrib.lite.Interpreter(model_path='/home/aldo/Documents/data-cic/tflite_models/vgg_16_2/tflite_vgg16_2.tflite')
# interpreter = tf.contrib.lite.Interpreter(model_path='/home/aldo/Documents/data-cic/tflite_models/vgg_13_bn/tflite_vgg13_bn.tflite')
# interpreter = tf.contrib.lite.Interpreter(model_path='/home/aldo/Documents/data-cic/tflite_models/mobilenetv1/tflite_mobilenetv1.tflite')
# interpreter = tf.contrib.lite.Interpreter(model_path='/home/aldo/Documents/data-cic/tflite_models/ssdlitemnetv1/ssdlitemnetv1.tflite')
# interpreter = tf.contrib.lite.Interpreter(model_path='/home/aldo/Documents/data-cic/tflite_models/ssdlitemnetv2/ssdlitemnetv2.tflite')
# interpreter = tf.contrib.lite.Interpreter(model_path='/home/aldo/Documents/data-cic/tflite_models/13_17/conv13_17.tflite')

# interpreter = tf.contrib.lite.Interpreter(model_path='/home/aldo/Documents/data-cic/tflite_models/trade-off/tflite_files/alpha_35_exp_1_f1.tflite')
interpreter = tf.contrib.lite.Interpreter(model_path='/home/aldo/Documents/data-cic/tflite_models/PPN/PPN_alpha_1_exp_1_f1.tflite')


interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test model on random input data.
# input_shape = input_details[0]['shape']
input_data = np.array(image_test, dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data[0][500:505])

/home/aldo/anaconda3/envs/test2/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  
/home/aldo/anaconda3/envs/test2/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  import sys
/home/aldo/anaconda3/envs/test2/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  


[[ 9.98484194e-01  1.51586067e-03  1.39730918e+00  8.92679989e-01
  -4.07404137e+00 -5.81618249e-01  9.33333337e-01  3.46666664e-01
   1.50000006e-01  1.50000006e-01  1.00000001e-01  1.00000001e-01
   2.00000003e-01  2.00000003e-01]
 [ 9.98233438e-01  1.76650845e-03  1.70205861e-01  3.97505909e-02
  -1.91881573e+00  1.40886414e+00  9.33333337e-01  3.46666664e-01
   2.22485960e-01  2.22485960e-01  1.00000001e-01  1.00000001e-01
   2.00000003e-01  2.00000003e-01]
 [ 9.97911990e-01  2.08804966e-03  3.00758898e-01  4.28533614e-01
   7.22486794e-01  1.19467163e+00  9.33333337e-01  3.46666664e-01
   1.06066018e-01  2.12132037e-01  1.00000001e-01  1.00000001e-01
   2.00000003e-01  2.00000003e-01]
 [ 9.97967303e-01  2.03263457e-03  1.13933110e+00  1.12009835e+00
  -1.39048755e+00  1.22335780e+00  9.33333337e-01  3.46666664e-01
   1.22474484e-01  1.83711737e-01  1.00000001e-01  1.00000001e-01
   2.00000003e-01  2.00000003e-01]
 [ 9.98313308e-01  1.68675557e-03 -8.49301666e-02 -1.06040180e+00
  